In [ ]:
import tensorflow as tf
import pandas as pd
import json
import configparser
from pathlib import Path
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

configs = configparser.ConfigParser()
configs.read("../../configuracoes.conf",encoding='utf-8')
data_folder = Path(configs["DEFAULT"]["folder_path"])
path = data_folder / configs["DEFAULT"]["file"]
objeto = configs["DEFAULT"]["objeto"]
regiao = configs["DEFAULT"]["regiao"]

look_back = 12 #Quantidade de meses anteriores usado para prever
forecast_horizon = int(configs["DEFAULT"]["meses"]) #Quantidade de meses previstos

# Prever Cesta

In [ ]:
#Carrega os dados
cesta = pd.read_excel(path)
cesta.drop(['ano'], axis=1, inplace=True)
cesta = cesta / 1000
cesta = cesta.iloc[::, ::]

#Cria sequencias de tempo de tamanho (meses anteriores + meses a serem previstos)
for n_step in range(1, look_back + forecast_horizon):
    cesta[f'preco t(h + {n_step})'] = cesta['preco'].shift(-n_step).values
cesta.dropna(inplace=True)
cesta.reset_index(drop=True, inplace=True)

#Divide os dados em Treino e Teste
X_train = cesta.iloc[::, :look_back:].values
y_train = cesta.iloc[::, look_back::].values
X_val = cesta.iloc[-1::, -12::].values

#remodela os dados para formato (batches, sequencia de tempo, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

#Cria o modelo
model=Sequential()
model.add(LSTM(32, stateful=True, input_shape=(look_back, 1), return_sequences=True, batch_input_shape=(1, look_back, 1)))
model.add(LSTM(32,stateful=True))
model.add(Dense(forecast_horizon))
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))

#Treina e prever (forecast_horizon) quantidade de tempo
history = model.fit(X_train, y_train, epochs= 150, batch_size=1, shuffle=False)
forecast = model.predict(X_val, batch_size=1)

results = [value for value in forecast[0]]
objeto = objeto.replace(" ",'_')
output = {objeto.lower():str(results)}
file_name = f"previsao_LSTM_{objeto.lower()}_{regiao.lower()}.json"

full_url = f"../../output/previsoes_produtos/{regiao}/{file_name}"
os.makedirs(os.path.dirname(full_url), exist_ok=True)
with open(full_url,"w") as file:
    json.dump(output,file,ensure_ascii=False)
print("Arquivo com resultados gerados")

# Prever Produtos

In [ ]:
path_inicial = "../../data/datasets_produtos/"

regioes = ['ilheus','itabuna']
produtos = ['acucar', 'arroz', 'banana', 'cafe', 'carne', 'farinha', 'feijao', 'leite', 
            'manteiga', 'oleo', 'pao', 'tomate']

for regiao in regioes:
    for objeto in produtos:
        #Carrega os dados
        path = f'{path_inicial}{regiao}/{objeto}_{regiao}.xlsx'
        cesta = pd.read_excel(path)
        cesta.drop(['ano'], axis=1, inplace=True)
        cesta = cesta / 1000
        cesta = cesta.iloc[::]

        #Cria sequencias de tempo de tamanho (meses anteriores + meses a serem previstos)
        for n_step in range(1, look_back + forecast_horizon):
            cesta[f'preco t(h + {n_step})'] = cesta['preco'].shift(-n_step).values
        cesta.dropna(inplace=True)
        cesta.reset_index(drop=True, inplace=True)

        #Divide os dados em Treino e Teste
        X_train = cesta.iloc[::, :look_back:].values
        y_train = cesta.iloc[::, look_back::].values
        X_val = cesta.iloc[-1::, -12::].values

        #remodela os dados para formato (batches, sequencia de tempo, features)
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

        #Cria o modelo
        model=Sequential()
        model.add(LSTM(32, stateful=True, input_shape=(look_back, 1), return_sequences=True, batch_input_shape=(1, look_back, 1)))
        model.add(LSTM(32,stateful=True))
        model.add(Dense(forecast_horizon))
        model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))

        #Treina e prever (forecast_horizon) quantidade de tempo
        history = model.fit(X_train, y_train, epochs= 150, batch_size=1, shuffle=False)
        forecast = model.predict(X_val, batch_size=1)

        results = [value for value in forecast[0]]
        objeto = objeto.replace(" ", '_')
        output = {objeto.lower():str(results)}
        file_name = f"previsao_LSTM_{objeto.lower()}_{regiao.lower()}.json"
       
        full_url = f"../../output/previsoes_produtos/{regiao}/{file_name}"
        os.makedirs(os.path.dirname(full_url), exist_ok=True)
        with open(full_url,"w") as file:
            json.dump(output,file,ensure_ascii=False)
        print("Arquivo com resultados gerados")